In [1]:
# Dependencies 
import pandas as pd
import os
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [3]:
# Import alternate championhsip points tables
points_table_formula1 = pd.read_csv('points_systems_csv/2024_formula1_race_points_table.csv')
points_table_imsa = pd.read_csv('points_systems_csv/2024_imsa_race_points_table.csv')
points_table_imsa_qual = pd.read_csv('points_systems_csv/2024_imsa_qualifying_points_table.csv')

In [4]:
all_files_path = 'race_results_html/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

# Initialize a list to contain the race results 
race_result_data = []
i = 0

for race_result_file in race_results_files:
# Get City / Race Name
    i += 1
    j = 0
    city = race_result_file.split('.')[0]
    city = city[18:]
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')

# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        j += 1
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        if j == 1 and i == 1:
            row_data_with_city = row_data + ['City']
            race_result_data.append(row_data_with_city)
        elif j != 1:
            row_data_with_city = row_data + [city]
            race_result_data.append(row_data_with_city)
        else:
            pass

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_result_df['Rank'] = race_result_df['Rank'].astype('int')
    race_result_df['Starts'] = race_result_df['Starts'].astype('int')
    race_result_df['Laps'] = race_result_df['Laps'].astype('int')
    race_result_df['Laps Led'] = race_result_df['Laps'].astype('int')
    race_result_df['Points'] = race_result_df['Points'].astype('int')
    race_result_df['Pit stop'] = race_result_df['Pit stop'].astype('int64')

    race_result_df = pd.merge(race_result_df, points_table_formula1, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa_qual, on='Starts', how='inner')
    race_result_df['Points (IMSA Scoring)'] = race_result_df['Points (IMSA Scoring)']+race_result_df['Points (IMSA Scoring) - Qualifying']
    race_result_df = race_result_df.drop(columns=['Points (IMSA Scoring) - Qualifying'])

race_result_df.sample(46)

,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Points (F1 Scoring),Points (IMSA Scoring)
322,19,Alexander Rossi,7,13,250,02:13:36.2301,250,Contact,12,140.340,4,gateway,0,138
269,20,Christian Rasmussen,20,20,85,01:46:06.6539,85,Running,10,117.225,3,indyrc,0,121
375,18,Sting Ray Robb,41,21,221,01:53:03.4362,221,Contact,12,119.045,4,milwaukee2,0,140
228,12,Christian Rasmussen,33,24,200,02:58:54.7313,200,Running,18,167.680,6,indy500,0,197
250,1,Alex Palou,10,1,85,01:45:27.2320,85,Running,54,117.956,3,indyrc,25,385
181,20,David Malukas,66,25,109,01:55:56.2806,109,Running,10,110.788,3,portland,0,116
84,4,Scott Dixon,9,3,250,01:26:41.2798,250,Running,32,154.693,2,iowa2,12,310
387,3,Scott Dixon,9,15,85,01:39:29.3973,85,Running,36,91.553,2,toronto,15,316
19,20,Scott McLaughlin,3,4,99,02:06:40.7466,99,Running,10,77.134,2,detroit,0,138
49,23,Christian Lundgaard,45,7,84,01:42:34.2538,84,Running,7,96.701,3,longbeach,0,104
